<font face="Calibri" size="2"> <i>eSBAE - Notebook Series - Part 3, version 0.4, March 2023. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![title](images/header.png)

# II - eSBAE Time-Series Extraction
### Extract various time-series data for large sets of points from Google Earth Engine
-------

This notebook takes you through the process of extracting time-series for a set of points using [Google's Earth Engine](https://earthengine.google.com/). The script is optimized to deal with thousands of points and will use parallelization to efficiently extract the information from the platform.

**You will need**:
- a valid Earth Engine account ([sign up here](https://code.earthengine.google.com/register))
- an uploaded table of points (Feature Collection) 
- the table needs a unique point identifier (Point ID)

**You should be aware, that:** 

- This notebook does **not need huge resources**, as processing is done on the platform. A **m2 instance** or lower is sufficient.  
- The extraction can take up to days (>100000 points). If you are on SEPAL, make use of the **"keep instance running"** option within the user report dashboard. However, **do not forget** to shut down your machine once processing finished. 
- A logfile is created within your tmp-folder. Interruption of connectivity to the SEPAL server may lead to block the output of the Jupyter notebook. **This does not mean the processing stopped.** You can see in esbae_log_(time) if the processing is still on going. 
- You can restart the kernel and execute all cells, and extraction will **start where it stopped**. This is also valid, if your instance has been shut down before processing was completely finished.

### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [ ]:
import ee
from sampling_handler.time_series import ts_extract

# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [ ]:
# project name (will be used for folder creation in your hone folder)
project = 'myESBAE'

# define the column name foe your unique point identifier within your Feature collection
point_id_name = "point_id"

# Point Feature Collection from where to extract points
points_fc = ee.FeatureCollection(
    "users/<username>/<points_fc_name>"
).sort(point_id_name)

#### 2c - Time-series parameter settings

In [ ]:
# start of calibration period (mainly for bfast)
start_calibration = "2010-01-01"  # YYYY-MM-DD format

# Actual period of interest, i.e. monitoring period
start_monitor =     "2015-01-01"  # YYYY-MM-DD format
end_monitor =       "2023-01-01"  # YYYY-MM-DD format

# Directory where output and temp files will go
outdir = f"/home/sepal-user/{project}/03_time_series_extraction"

# select the bands to extract
bands = [
    #'blue', 
    'green', 'red', 'nir', 'swir1', 'swir2',           # reflectance bands
    'ndfi', 'ndmi', #'mndwi', 'nbr', 'ndvi'            # available ratios
    'brightness', 'greenness', 'wetness'               # tasseled cap
]

# select the band for univariate ts-analysis (has to be inside bands list)
ts_band = 'ndfi'

# select the resolution to which the satellite data will be resized.
scale = 30  # in meters (70 m is half ha, relates to FAO forest definition)

### DO NOT CHANGE YET ###
satellite='Landsat'  # this is going to be Surface Reflactance, Collection 2, Tier 1 data only
max_cloud_cover = 75  # in percentage (0-100)

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [ ]:
# landsat related parameters
lsat_params = {
    'l9':     True,
    'l8':     True,
    'l7':     True,
    'l5':     True,
    'l4':     True,
    'brdf':   True,
    'bands':  bands,
    'max_cc': max_cloud_cover
}


### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir':                         outdir,
    'workers':                          10,
    'max_points_per_chunk':             75,
    'grid_size_levels':                 [0.1, 0.05, 0.025],  # definition of chunk sizes in degrees  
    'lsat_params':                      lsat_params,
    'ts_params': {
        'start_calibration':            start_calibration,
        'start_monitor':                start_monitor,
        'end_monitor':                  end_monitor,
        'point_id':                     point_id_name,
        'bands':                        bands,
        'ts_band':                      ts_band,
        'satellite':                    satellite,
        'scale':                        scale,
        'max_cc':                       max_cloud_cover,
    },    
}

### 4 - Run the time-series data extraction

**Execute only**

In [ ]:
ts_extract.extract(points_fc, config_dict)